# A Covid-19 dashboard comparing various parameters based on Public Health England data

In [1]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [2]:
def create_api_data():    
    
    print("I'm downloading data from the API...")
    filters = [
        "areaType=overview"
    ]

    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "hospital": "newAdmissions",
        "deaths": "cumDeaths28DaysByDeathDateRate",
        "males": "maleCases",
        "females": "femaleCases"
    }

    api = Cov19API(filters=filters, structure=structure)
    timeseries = api.get_json()

    filters = [
        'areaType=nation',
        'areaName=England'
    ]

    structure = {
        "males": "maleCases",
        "females": "femaleCases"
    }

    api = Cov19API(filters=filters, structure=structure)
    agedistribution = api.get_json()

    filters = [
        'areaType=nation',
        'areaName=England'
    ]

    structure = {
        "date": "date",
        "newAdmissions": "newAdmissions",
        "covidOccupiedMVBeds": "covidOccupiedMVBeds"
    }

    api = Cov19API(filters=filters, structure=structure)
    ventilator = api.get_json()

    filters = [
        'areaType=region',
        'areaName=London'
    ]

    structure = {
        "date": "date",
        "deaths": "newDeaths28DaysByDeathDate",
        "cases": "newCasesBySpecimenDate"
    }

    api = Cov19API(filters=filters, structure=structure)
    london_cases_deaths = api.get_json()

    with open("timeseries.json", "wt") as OUTF:
        json.dump(timeseries, OUTF)

    with open("agedistribution.json", "wt") as OUTF:
        json.dump(agedistribution, OUTF)

    with open("ventilator.json", "wt") as OUTF:
        json.dump(ventilator, OUTF)

    with open("london_cases_deaths.json", "wt") as OUTF:
        json.dump(london_cases_deaths, OUTF)

    print("...all done.")

In [3]:
def access_api(button):
    create_api_data()

apibutton = wdg.Button(
        description='Refresh data',
        disabled=False,
        button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click to download current Public Health England data',
        icon='download'  # (FontAwesome names without the `fa-` prefix)
    )

apibutton.on_click(access_api)

display(apibutton)

Button(button_style='success', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Clic…

In [4]:
with open("london_cases_deaths.json", "rt") as INFILE:
    data = json.load(INFILE)

dataL = data["data"]

datesL = [dictionary["date"] for dictionary in dataL]
datesL.sort()

def parse_date(datastring):
    return pd.to_datetime(datastring, format="%Y-%m-%d")

startdateL = parse_date(datesL[0])
enddateL = parse_date(datesL[-1])

index = pd.date_range(startdateL, enddateL, freq="D")
london_cases_deathsdf = pd.DataFrame(index=index, columns=["deaths", "cases"])

for entry in dataL:
    date = parse_date(entry['date'])
    for column in ["deaths", "cases"]:
        if pd.isna(london_cases_deathsdf.loc[date, column]):
            value = float(entry[column]) if entry[column] != None else 0.0
            london_cases_deathsdf.loc[date, column] = value

london_cases_deathsdf.fillna(0.0, inplace=True)

london_cases_deathsdf.to_pickle("london_cases_deathsdf.pkl")
london_cases_deathsdf = pd.read_pickle("london_cases_deathsdf.pkl")

londoncols = wdg.SelectMultiple(
    options=['cases', 'deaths'],
    value=['cases', 'deaths'],
    rows=2,
    layout={'width': 'max-content'},
    description='Options',
    disabled=False
)

def london_graph(graphcolumns):
    ncols = len(graphcolumns)
    if ncols > 0:
        london_cases_deathsdf.plot(kind='bar', y=list(graphcolumns))
        plt.xticks([])
        plt.xlabel('Time')
        plt.title("Comparison of new cases and new deaths in London")
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

output_london_bar = wdg.interactive_output(london_graph, {'graphcolumns': londoncols})

display(londoncols, output_london_bar)

SelectMultiple(description='Options', index=(0, 1), layout=Layout(width='max-content'), options=('cases', 'dea…

Output()

In [5]:
with open("london_cases_deaths.json", "rt") as INFILE:
    data = json.load(INFILE)

dataL = data["data"]

datesL = [dictionary["date"] for dictionary in dataL]
datesL.sort()

def parse_date(datastring):
    return pd.to_datetime(datastring, format="%Y-%m-%d")

startdateL = parse_date(datesL[0])
enddateL = parse_date(datesL[-1])

index = pd.date_range(startdateL, enddateL, freq="D")
london_cases_deathsdf = pd.DataFrame(index=index, columns=["deaths", "cases"])

for entry in dataL:
    date = parse_date(entry['date'])
    for column in ["deaths", "cases"]:
        if pd.isna(london_cases_deathsdf.loc[date, column]):
            value = float(entry[column]) if entry[column] != None else 0.0
            london_cases_deathsdf.loc[date, column] = value

london_cases_deathsdf.fillna(0.0, inplace=True)

london_cases_deathsdf.to_pickle("london_cases_deathsdf.pkl")

london_cases_deathsdf = pd.read_pickle("london_cases_deathsdf.pkl")

london = wdg.SelectMultiple(
    options=['cases', 'deaths'],
    value=['cases', 'deaths', ],
    rows=2,
    description='Stats:',
    disabled=False
)

scale = wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls = wdg.HBox([london, scale])

def london_graph(gcols, gscale):
    if gscale == 'linear':
        logscale = False
    else:
        logscale = True
    ncols = len(gcols)
    if ncols > 0:
        london_cases_deathsdf[list(gcols)].plot(logy=logscale)
        plt.title("Comparison of cases and deaths in London")
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

output_london_linear = wdg.interactive_output(london_graph, {'gcols': london, 'gscale': scale})

display(controls, output_london_linear)

Output()

In [6]:
with open("ventilator.json", "rt") as INFILE:
    data = json.load(INFILE)

datav = data["data"]

datesv = [dictionary["date"] for dictionary in datav]
datesv.sort()

def parse_date(datastring):
    return pd.to_datetime(datastring, format="%Y-%m-%d")

startdatev = parse_date(datesv[0])
enddatev = parse_date(datesv[-1])

index = pd.date_range(startdatev, enddatev, freq="D")
ventilatordf = pd.DataFrame(index=index, columns=["newAdmissions", "covidOccupiedMVBeds"])

for entry in datav:
    date = parse_date(entry['date'])
    for column in ["newAdmissions", "covidOccupiedMVBeds"]:
        if pd.isna(ventilatordf.loc[date, column]):
            value = float(entry[column]) if entry[column] != None else 0.0
            ventilatordf.loc[date, column] = value

ventilatordf.fillna(0.0, inplace=True)

ventilatordf.to_pickle("ventilatordf.pkl")
ventilatordf = pd.read_pickle("ventilatordf.pkl")

ventilatorcols = wdg.SelectMultiple(
    options=['newAdmissions', 'covidOccupiedMVBeds'],
    value=['newAdmissions', 'covidOccupiedMVBeds'],
    rows=2,
    description='Options',
    disabled=False
)

def ventilator_graph(graphcolumns):
    ncols = len(graphcolumns)
    if ncols > 0:
        ventilatordf.plot(kind='bar', y=list(graphcolumns))
        plt.xticks([])
        plt.xlabel('Time')
        plt.title("Comparison of new admissions and occupied mechanical ventilators")
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

output_ventilators = wdg.interactive_output(ventilator_graph, {'graphcolumns': ventilatorcols})

display(ventilatorcols, output_ventilators)

SelectMultiple(description='Options', index=(0, 1), options=('newAdmissions', 'covidOccupiedMVBeds'), rows=2, …

Output()

In [7]:
with open("timeseries.json", "rt") as INFILE:
    data = json.load(INFILE)

datalist = data["data"]

dates = [dictionary["date"] for dictionary in datalist]
dates.sort()

def parse_date(datastring):
    return pd.to_datetime(datastring, format="%Y-%m-%d")

startdate = parse_date(dates[0])
enddate = parse_date(dates[-1])

index = pd.date_range(startdate, enddate, freq="D")
timeseriesdf = pd.DataFrame(index=index, columns=["cases", "hospital", "deaths"])

for entry in datalist:
    date = parse_date(entry['date'])
    for column in ['cases', 'hospital', 'deaths']:
        if pd.isna(timeseriesdf.loc[date, column]):
            value = float(entry[column]) if entry[column] != None else 0.0
            timeseriesdf.loc[date, column] = value

timeseriesdf.fillna(0.0, inplace=True)

timeseriesdf.to_pickle("timeseriesdf.pkl")
timeseriesdf = pd.read_pickle("timeseriesdf.pkl")

series = wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale = wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls = wdg.HBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale == 'linear':
        logscale = False
    else:
        logscale = True
    ncols = len(gcols)
    if ncols > 0:
        timeseriesdf[list(gcols)].plot(logy=logscale)
        plt.title("Comparison of cases, hospital cases and deaths")
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

output_timeseries = wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

display(controls, output_timeseries)

Output()

In [8]:

with open("agedistribution.json", "rt") as INFILE:
    data = json.load(INFILE)

datadic = data['data'][0]
males = datadic['males']
females = datadic['females']

ageranges = [x['age'] for x in males]

def min_age(agerange):
    agerange = agerange.replace('+', '')
    start = agerange.split('_')[0]
    return int(start)

ageranges.sort(key=min_age)

age_df = pd.DataFrame(index=ageranges, columns=['males', 'females', 'total'])

for entry in males:
    ageband = entry['age']
    age_df.loc[ageband, 'males'] = entry['value']

for entry in females:
    ageband = entry['age']
    age_df.loc[ageband, 'females'] = entry['value']

age_df['total'] = age_df['males'] + age_df['females']

age_df.to_pickle("agedf.pkl")
age_df = pd.read_pickle("agedf.pkl")

agecols = wdg.SelectMultiple(
    options=['males', 'females', 'total'],
    value=['males', 'females'],
    rows=3,
    description='Sex',
    disabled=False
)

def age_graph(graphcolumns):
    ncols = len(graphcolumns)
    if ncols > 0:
        age_df.plot(kind='bar', y=list(graphcolumns))
        plt.title("Comparison of cases in males and females across age groups")
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

output_age = wdg.interactive_output(age_graph, {'graphcolumns': agecols})

display(agecols, output_age)

SelectMultiple(description='Sex', index=(0, 1), options=('males', 'females', 'total'), rows=3, value=('males',…

Output()